# Examples of Undetected Tiger

In [1]:
import boto3
import matplotlib.pyplot as plt
import json
import numpy as np
from PIL import Image, ImageDraw
import os

In [2]:
s3 = boto3.resource('s3')
bucket_name = 'calvinandpogs-ee148'
bucket = s3.Bucket(bucket_name)

In [3]:
with open('./ATRWEvalScript/annotations/detect_tiger02_test.json') as f:
    ground_truth = json.load(f)

In [4]:
best_preds_path = 'atrw/out/detection/yolov5/test/05-26-2021-22-59-19/runs/best_predictions.json' #05-27-2021-07-47-09/ no access
preds_obj = bucket.Object(best_preds_path).get()['Body'].read()
best_preds = json.loads(preds_obj.decode("utf-8"))

img_path = 'atrw/detection/test/'

In [ ]:
test_dir ='atrw/out/detection/yolov5/test/05-25-2021-22-04-31/runs/labels/'

for obj in bucket.objects.filter(Prefix=test_dir):
    key = obj.key.strip('.txt')[-4:]
    image_id = int(key)
    
    pred_count = obj.get()['Body'].read().decode("utf-8").count('\n')
    img_tigers_bbox = []
    for tiger in ground_truth['annotations']:
        if tiger['image_id'] == image_id:
            #print(img_tigers_bbox, tiger['bbox'])
            for pred in best_preds:
                if pred['image_id'] == image_id:
                    img_tigers_bbox.append(pred['bbox'])
            #img_tigers_bbox.append(tiger['bbox'])

    if len(tiger) > pred_count:
        image_id_name = key + '.jpg'
        img_path = os.path.join(img_path, image_id_name)
        #print(img_path)
        img_obj = bucket.Object(img_path).get()['Body'].read()

        I = np.asarray(Image.open(img_obj))
        img = Image.fromarray(I, 'RGB')
        draw = ImageDraw.Draw(img)
        for coors in img_tigers_bbox:
            draw.rectangle(coors)
        img.show()

        FN += 1
        if FN > 10: 
            break

# for every image in test set, compare w/ ground truth
# determine whether the number identified is less (1 of many cases for FN)
# extract the image and plot the bounding boxes  hw #3
# export image for report

In [ ]:
# temp: plotting bounding boxes over imported background images
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import base64

image1_filename = 'img_lights.png'
polar_light = base64.b64encode(open(image1_filename, 'rb').read())
image2_filename = 'img_snowtops.png'
snow = base64.b64encode(open(image2_filename, 'rb').read())

fig = make_subplots(
    rows=2, cols=1, subplot_titles=("Title 1", "Title 2"),
    vertical_spacing=0.075
)

fig.add_trace(go.Scatter(x= [1, 1.75, 2.5, 3.5], y=[-1, 3, 0, 3,  5],
                         mode='markers',
                         marker_size=20,
                         marker_color="#ffd700"),
              row=1, col=1)

fig.add_trace(go.Scatter(x= [1, 1.75, 2.5, 3.5], y=[4, 2, 6, 3,  5], mode='markers',
                         marker_symbol='triangle-down',
                         marker_size=20, marker_color='red'),
              row=2, col=1)
fig.update_layout(width=850, height=700,
                images= [dict(
                    source='data:image/png;base64,{}'.format(polar_light.decode()),
                    xref="paper", yref="paper",
                    x=0, y=1,  #position of the upper left corner of the image in subplot 1,1
                    sizex=1, sizey=0.46, #sizex, sizey are set by trial and error
                    xanchor="left",
                    yanchor="top",
                    sizing="stretch",
                    layer="below"),
                    dict(
                    source='data:image/png;base64,{}'.format(snow.decode()),
                    xref="paper", yref="paper",
                    x=0, y=0.46,  #position of the upper left corner of the image in subplot 2,1
                    sizex=1, sizey=0.46,
                    xanchor="left",
                    yanchor="top",
                    sizing="stretch",
                    layer="below")    ])

fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)